# Testing Kernels/Covariance functions

One can check if the derivatives look right via [desmos](https://www.desmos.com/3d/urvu7exzyx)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from gaussian_process import GaussianProcess
from gaussian_process import GPfunctions
from gaussian_process.kernels import *
import itertools

# TODO: Share y-axes between graphs or have y-axis on right graphs labeled

In [ ]:
# Setup

X = np.linspace(start=-3.0, stop=3.0, num=1_000)

objectiveFunction = lambda x: -x * np.sin(x)
objectiveFunctionDerivative = lambda x: -x * np.cos(x) - np.sin(x)

y = objectiveFunction(X)
g = objectiveFunctionDerivative(X)

rng = np.random.default_rng(1)
training_indices = rng.choice(np.arange(y.size), size=6, replace=False)
X_train, y_train, g_train = (
    X[training_indices],
    y[training_indices],
    g[training_indices],
)

In [ ]:
def plot_kernel(ax, kernel: Kernel, plotDerivatives: bool = True):
    offset = 1.0
    ax.plot(X, [kernel(offset, x) for x in X], label=f"k({offset},x)")
    ax.plot(X, [kernel(x, offset) for x in X], label=f"k(x,{offset})")
    if plotDerivatives:
        ax.plot(
            X, [kernel.derivative_rhs(offset, x) for x in X], label=f"dk({offset},x)"
        )
        ax.plot(
            X, [kernel.derivative_lhs(x, offset) for x in X], label=f"dk(x,{offset})"
        )
        ax.plot(
            X,
            [kernel.derivative_lhsrhs(offset, x) for x in X],
            label=f"ddk({offset},x)",
        )
        ax.plot(
            X,
            [kernel.derivative_lhsrhs(x, offset) for x in X],
            label=f"ddk(x,{offset})",
        )


def plot_label(ax, title):
    ax.legend()
    ax.grid()
    ax.set_title(title)
    ax.set(xlabel="$x$", ylabel="$f(x)$")
    ax.label_outer()

## Squared exponential

In [ ]:
ls = [0.5, 1.0, 2.0]

for l in ls:
    kernel = SquaredExponentialKernel(l)

    gp = GaussianProcess(kernel, X_train, y_train, g_train)
    mean, variance = gp(X)

    fig, (ax1, ax2) = plt.subplots(1, 2)
    plot_kernel(ax1, kernel)
    GPfunctions.plot_objective(ax2, X, y, X_train, y_train)
    GPfunctions.plot_gp(ax2, X, mean, variance, label="GP")
    plot_label(ax1, title="Kernel")
    plot_label(ax2, title="GP")
    fig.suptitle(f"{kernel.__class__.__name__}, l={l}")
    fig.set_figwidth(15)
    plt.show()

    assert kernel.is_symmetric
    assert kernel.is_positive_definite
    assert kernel.is_covariance_function

## Exponential

In [ ]:
ls = [0.5, 1.0, 2.0]

for l in ls:
    kernel = ExponentialKernel(l)

    # Results with g_train are weird, because derivative at 0 is NaN or infinity
    gp = GaussianProcess(kernel, X_train, y_train)
    mean, variance = gp(X)

    fig, (ax1, ax2) = plt.subplots(1, 2)
    plot_kernel(ax1, kernel)
    GPfunctions.plot_objective(ax2, X, y, X_train, y_train)
    GPfunctions.plot_gp(ax2, X, mean, variance, label="GP")
    plot_label(ax1, title="Kernel")
    plot_label(ax2, title="GP")
    fig.suptitle(f"{kernel.__class__.__name__}, l={l}")
    fig.set_figwidth(15)
    plt.show()

    assert kernel.is_symmetric
    assert kernel.is_positive_definite
    assert kernel.is_covariance_function

## Gamma exponential

In [ ]:
gammas = [0.9, 1.0, 1.5, 2.0]
ls = [0.5, 1.0, 2.0]

for gamma, l in itertools.product(gammas, ls):
    kernel = GammaExponentialKernel(gamma=gamma, l=l)

    # Results with g_train are weird, because derivative at 0 is NaN or infinity
    gp = GaussianProcess(
        kernel, X_train, y_train, g_known=g_train if gamma == 2.0 else None
    )
    mean, variance = gp(X)

    fig, (ax1, ax2) = plt.subplots(1, 2)
    plot_kernel(ax1, kernel)
    GPfunctions.plot_objective(ax2, X, y, X_train, y_train)
    GPfunctions.plot_gp(ax2, X, mean, variance, label="GP")
    plot_label(ax1, title="Kernel")
    plot_label(ax2, title="GP")
    fig.suptitle(f"{kernel.__class__.__name__}, gamma={gamma}, l={l}")
    fig.set_figwidth(15)
    plt.show()

    assert kernel.is_symmetric
    assert kernel.is_positive_definite
    assert kernel.is_covariance_function

## Cubic

In [ ]:
kernel = CubicKernel()

gp = GaussianProcess(kernel, X_train, y_train, g_train)
mean, variance = gp(X)

fig, (ax1, ax2) = plt.subplots(1, 2)
plot_kernel(ax1, kernel)
GPfunctions.plot_objective(ax2, X, y, X_train, y_train)
GPfunctions.plot_gp(ax2, X, mean, variance, label="GP")
plot_label(ax1, title="Kernel")
plot_label(ax2, title="GP")
fig.suptitle(kernel.__class__.__name__)
fig.set_figwidth(15)
plt.show()

assert kernel.is_symmetric
assert not kernel.is_positive_definite
assert not kernel.is_covariance_function

## Rational quadratic

In [ ]:
alphas = [0.5, 1.0, 2.0]
ls = [0.5, 1.0, 2.0]

for alpha, l in itertools.product(alphas, ls):
    kernel = RationalQuadraticKernel(alpha=alpha, l=l)

    gp = GaussianProcess(kernel, X_train, y_train, g_train)
    mean, variance = gp(X)

    fig, (ax1, ax2) = plt.subplots(1, 2)
    plot_kernel(ax1, kernel)
    GPfunctions.plot_objective(ax2, X, y, X_train, y_train)
    GPfunctions.plot_gp(ax2, X, mean, variance, label="GP")
    plot_label(ax1, title="Kernel")
    plot_label(ax2, title="GP")
    fig.suptitle({f"{kernel.__class__.__name__}, alpha={alpha}, l={l}"})
    fig.set_figwidth(15)
    plt.show()

    assert kernel.is_symmetric
    assert kernel.is_positive_definite
    assert kernel.is_covariance_function

## Matern

In [ ]:
ls = [0.5, 1.0, 2.0]

for l in ls:
    kernel = Matern1_5Kernel(l=l)

    gp = GaussianProcess(kernel, X_train, y_train, g_train)
    mean, variance = gp(X)

    fig, (ax1, ax2) = plt.subplots(1, 2)
    plot_kernel(ax1, kernel)
    GPfunctions.plot_objective(ax2, X, y, X_train, y_train)
    GPfunctions.plot_gp(ax2, X, mean, variance, label="GP")
    plot_label(ax1, title="Kernel")
    plot_label(ax2, title="GP")
    fig.suptitle({f"{kernel.__class__.__name__}, l={l}"})
    fig.set_figwidth(15)
    plt.show()

    assert kernel.is_symmetric
    assert kernel.is_positive_definite
    assert kernel.is_covariance_function

for l in ls:
    kernel = Matern2_5Kernel(l=l)

    gp = GaussianProcess(kernel, X_train, y_train, g_train)
    mean, variance = gp(X)

    fig, (ax1, ax2) = plt.subplots(1, 2)
    plot_kernel(ax1, kernel)
    GPfunctions.plot_objective(ax2, X, y, X_train, y_train)
    GPfunctions.plot_gp(ax2, X, mean, variance, label="GP")
    plot_label(ax1, title="Kernel")
    plot_label(ax2, title="GP")
    fig.suptitle({f"{kernel.__class__.__name__}, l={l}"})
    fig.set_figwidth(15)
    plt.show()

    assert kernel.is_symmetric
    assert kernel.is_positive_definite
    assert kernel.is_covariance_function

## Polyharmonic Spline

In [ ]:
ks = [1, 2, 3, 4, 5, 6, 7]

for k in ks:
    kernel = PolyharmonicSplineKernel(k=k)

    gp = GaussianProcess(kernel, X_train, y_train, g_known=g_train if k != 2 else None)
    mean, variance = gp(X)

    fig, (ax1, ax2) = plt.subplots(1, 2)
    plot_kernel(ax1, kernel)
    GPfunctions.plot_objective(ax2, X, y, X_train, y_train)
    GPfunctions.plot_gp(ax2, X, mean, variance, label="GP")
    plot_label(ax1, title="Kernel")
    plot_label(ax2, title="GP")
    fig.suptitle({f"{kernel.__class__.__name__}, k={k}"})
    fig.set_figwidth(15)
    plt.show()

    assert kernel.is_symmetric
    assert not kernel.is_positive_definite
    assert not kernel.is_covariance_function

## Constant

In [ ]:
cs = [0.0, 0.1, 1.0, 2.0]

for c in cs:
    kernel = ConstantKernel(c=c)

    gp = GaussianProcess(kernel, X_train, y_train, f_noise=1.0)
    mean, variance = gp(X)

    fig, (ax1, ax2) = plt.subplots(1, 2)
    plot_kernel(ax1, kernel)
    GPfunctions.plot_objective(ax2, X, y, X_train, y_train)
    GPfunctions.plot_gp(ax2, X, mean, variance, label="GP")
    plot_label(ax1, title="Kernel")
    plot_label(ax2, title="GP")
    fig.suptitle({f"{kernel.__class__.__name__}, c={c}"})
    fig.set_figwidth(15)
    plt.show()

    assert kernel.is_symmetric
    assert kernel.is_positive_definite
    assert kernel.is_covariance_function

## Linear

In [ ]:
slopes = [0.1, 1.0, 2.0]

for a in slopes:
    kernel = KernelSum(LinearKernel(a=a), ConstantKernel(1.0))

    gp = GaussianProcess(kernel, X_train, y_train, f_noise=1.0)
    mean, variance = gp(X)

    fig, (ax1, ax2) = plt.subplots(1, 2)
    plot_kernel(ax1, kernel)
    GPfunctions.plot_objective(ax2, X, y, X_train, y_train)
    GPfunctions.plot_gp(ax2, X, mean, variance, label="GP")
    plot_label(ax1, title="Kernel")
    plot_label(ax2, title="GP")
    fig.suptitle({f"{kernel.__class__.__name__}, a={a}"})
    fig.set_figwidth(15)
    plt.show()

    assert kernel.is_symmetric
    assert kernel.is_positive_definite
    assert kernel.is_covariance_function

## Polynomial

In [ ]:
ps = [2, 3, 4]
sigmas = [0.0, 0.5, 1.0, 2.0]

for p, sigma in itertools.product(ps, sigmas):

    kernel = PolynomialKernel(p=p, sigma=sigma)

    gp = GaussianProcess(kernel, X_train, y_train, g_train, f_noise=0.1, g_noise=0.1)
    mean, variance = gp(X)

    fig, (ax1, ax2) = plt.subplots(1, 2)
    plot_kernel(ax1, kernel)
    GPfunctions.plot_objective(ax2, X, y, X_train, y_train)
    GPfunctions.plot_gp(ax2, X, mean, variance, label="GP")
    plot_label(ax1, title="Kernel")
    plot_label(ax2, title="GP")
    fig.suptitle({f"{kernel.__class__.__name__}, p={p}, sigma={sigma}"})
    fig.set_figwidth(15)
    plt.show()

    assert kernel.is_symmetric
    assert kernel.is_positive_definite
    assert kernel.is_covariance_function

## Brownian

In [ ]:
kernel = BrownianKernel()

gp = GaussianProcess(kernel, X_train, y_train, f_noise=1e-10)
mean, variance = gp(X)

fig, (ax1, ax2) = plt.subplots(1, 2)
plot_kernel(ax1, kernel)
GPfunctions.plot_objective(ax2, X, y, X_train, y_train)
GPfunctions.plot_gp(ax2, X, mean, variance, label="GP")
plot_label(ax1, title="Kernel")
plot_label(ax2, title="GP")
fig.suptitle({f"{kernel.__class__.__name__}, p={p}, sigma={sigma}"})
fig.set_figwidth(15)
plt.show()

assert kernel.is_symmetric
assert not kernel.is_positive_definite
assert not kernel.is_covariance_function

## Sinc

In [ ]:
kernel = SincKernel()

gp = GaussianProcess(kernel, X_train, y_train, g_train, g_noise=1e-10)
mean, variance = gp(X)

fig, (ax1, ax2) = plt.subplots(1, 2)
plot_kernel(ax1, kernel)
GPfunctions.plot_objective(ax2, X, y, X_train, y_train)
GPfunctions.plot_gp(ax2, X, mean, variance, label="GP")
plot_label(ax1, title="Kernel")
plot_label(ax2, title="GP")
fig.suptitle({f"{kernel.__class__.__name__}"})
fig.set_figwidth(15)
plt.show()

assert kernel.is_symmetric
assert not kernel.is_positive_definite
assert not kernel.is_covariance_function

## Product Kernel

In [ ]:
kernel = KernelProduct(PolynomialKernel(p=5), SquaredExponentialKernel())

gp = GaussianProcess(kernel, X_train, y_train, g_train)
mean, variance = gp(X)

fig, (ax1, ax2) = plt.subplots(1, 2)
plot_kernel(ax1, kernel)
GPfunctions.plot_objective(ax2, X, y, X_train, y_train)
GPfunctions.plot_gp(ax2, X, mean, variance, label="GP")
plot_label(ax1, title="Kernel")
plot_label(ax2, title="GP")
fig.suptitle({f"{kernel.__class__.__name__}"})
fig.set_figwidth(15)
plt.show()

assert kernel.is_symmetric
assert kernel.is_positive_definite
assert kernel.is_covariance_function

## Sum Kernel

In [ ]:
kernel = KernelSum(PolynomialKernel(p=5), SquaredExponentialKernel())

gp = GaussianProcess(kernel, X_train, y_train, g_train)
mean, variance = gp(X)

fig, (ax1, ax2) = plt.subplots(1, 2)
plot_kernel(ax1, kernel)
GPfunctions.plot_objective(ax2, X, y, X_train, y_train)
GPfunctions.plot_gp(ax2, X, mean, variance, label="GP")
plot_label(ax1, title="Kernel")
plot_label(ax2, title="GP")
fig.suptitle({f"{kernel.__class__.__name__}"})
fig.set_figwidth(15)
plt.show()

assert kernel.is_symmetric
assert kernel.is_positive_definite
assert kernel.is_covariance_function